# [Recurrent Neural Network for Multivariate Time Series with Missing Values](https://www.nature.com/articles/s41598-018-24271-9.pdf)

## 1. low-level GRU implementation [reference](https://gist.github.com/kmjjacobs/eab1e840aecf0ac232cc8370a9be9093)

In [1]:
import tensorflow as tf
import numpy as np

def weight_variable(shape,train=True):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial,trainable=train)
def bias_variable(shape,train=True):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial,trainable=train)

class GRU():
    
    def __init__(self, input_dimensions,hidden_size,time_steps, dtype=tf.float32):
        self.input_dimensions = input_dimensions
        self.hidden_size = hidden_size
        self.time_steps = time_steps
        
        # Weights for input vectors of shape (input_dimensions, hidden_size)
        self.Wr = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.input_dimensions, self.hidden_size), mean=0, stddev=0.01), name='Wr')
        self.Wz = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.input_dimensions, self.hidden_size), mean=0, stddev=0.01), name='Wz')
        self.Wh = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.input_dimensions, self.hidden_size), mean=0, stddev=0.01), name='Wh')
        
        # Weights for hidden vectors of shape (hidden_size, hidden_size)
        self.Ur = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size, self.hidden_size), mean=0, stddev=0.01), name='Ur')
        self.Uz = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size, self.hidden_size), mean=0, stddev=0.01), name='Uz')
        self.Uh = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size, self.hidden_size), mean=0, stddev=0.01), name='Uh')
        
        # Biases for hidden vectors of shape (hidden_size,)
        self.br = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='br')
        self.bz = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='bz')
        self.bh = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='bh')
        
        # Define the input layer placeholder
        self.input_holder = tf.placeholder(dtype=dtype, shape=(None, time_steps, input_dimensions), name='input')
        
        
        # Put the time-dimension upfront for the scan operator
        self.x_t = tf.transpose(self.input_holder, [1, 0, 2], name='x_t')
        
        # A little hack (to obtain the same shape as the input matrix) to define the initial hidden state h_0
        self.h_0 = tf.matmul(self.x_t[0, :, :], tf.zeros(dtype=dtype, shape=(input_dimensions, hidden_size)), name='h_0')
        
        # Perform the scan operator
        self.h_t_transposed = tf.scan(self.forward_pass, self.x_t, initializer=self.h_0, name='h_t_transposed')
        
        # Transpose the result back
        self.h_t = tf.transpose(self.h_t_transposed, [1, 0, 2], name='h_t')

    def forward_pass(self, h_tm1, x_t):
        """Perform a forward pass.
        
        Arguments
        ---------
        h_tm1: np.matrix
            The hidden state at the previous timestep (h_{t-1}).
        x_t: np.matrix
            The input vector.
        """
        # Definitions of update gate
        z_t = tf.sigmoid(tf.matmul(x_t, self.Wz) + tf.matmul(h_tm1, self.Uz) + self.bz)
        # Definition of reset gate
        r_t = tf.sigmoid(tf.matmul(x_t, self.Wr) + tf.matmul(h_tm1, self.Ur) + self.br)
        
        # Definition of candidate actvation
        h_proposal = tf.tanh(tf.matmul(x_t, self.Wh) + tf.matmul(tf.multiply(r_t, h_tm1), self.Uh) + self.bh)
        
        # Compute the actual next hidden state
        h_t = tf.multiply(1 - z_t, h_tm1) + tf.multiply(z_t, h_proposal)
        
        return h_t
        

## 2. GRU implementation test ([data produce](https://github.com/MorvanZhou/tutorials/blob/master/tensorflowTUT/tf20_RNN2.2/full_code.py))

In [2]:
import matplotlib.pyplot as plt
%matplotlib qt5

BATCH_START = 0
TIME_STEPS = 20
BATCH_SIZE = 50
INPUT_SIZE = 1
OUTPUT_SIZE = 1
HIDDEN_SIZE = 12
LR = 0.006

tf.reset_default_graph()

def get_batch():
    global BATCH_START, TIME_STEPS
    # xs shape (50batch, 20steps)
    xs = np.arange(BATCH_START, BATCH_START+TIME_STEPS*BATCH_SIZE).reshape((BATCH_SIZE, TIME_STEPS)) / (10*np.pi)
    seq = np.sin(xs)
    res = np.cos(xs)
    BATCH_START += TIME_STEPS
    # plt.plot(xs[0, :], res[0, :], 'r', xs[0, :], seq[0, :], 'b--')
    # plt.show()
    # returned seq, res and xs: shape (batch, step, input)
    return [seq[:, :, np.newaxis], res[:, :, np.newaxis], xs]

model = GRU(input_dimensions=INPUT_SIZE , hidden_size=HIDDEN_SIZE, time_steps=TIME_STEPS )
output_holder = tf.placeholder(tf.float32,[None,TIME_STEPS,OUTPUT_SIZE])
w_out = weight_variable([HIDDEN_SIZE,OUTPUT_SIZE])
b_out = bias_variable([OUTPUT_SIZE])

h = tf.reshape(model.h_t,(-1,HIDDEN_SIZE))
predict = tf.matmul(h,w_out)+b_out
predict = tf.reshape(predict,(-1,TIME_STEPS,OUTPUT_SIZE))
loss = tf.reduce_mean((predict-output_holder)**2)
train_Op = tf.train.AdamOptimizer(learning_rate=LR).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())


plt.ion()
plt.show()
for i in range(300):
        seq, res, xs = get_batch()
        
        feed_dict = {
                    model.input_holder: seq,
                    output_holder: res
            }


        _, loss_, pred = sess.run(
            [train_Op, loss,predict],
            feed_dict=feed_dict)
        if i%20==0:
            print('loss: %g'%(loss_))
        plt.plot(xs[0, :], res[0].flatten(), 'r', xs[0, :], pred.flatten()[:TIME_STEPS], 'b--')
        plt.ylim((-1.2, 1.2))
        plt.draw()
        plt.pause(0.3)

loss: 0.511239
loss: 0.501083
loss: 0.459234
loss: 0.440753
loss: 0.418487
loss: 0.360051
loss: 0.372068
loss: 0.387811
loss: 0.315595
loss: 0.298827
loss: 0.310882
loss: 0.303998
loss: 0.262455
loss: 0.274973
loss: 0.296207


## 3. Literature model implementation with little modification for fitting transportation problem

### 3.1 Data preparation

In [4]:
import numpy as np
import tensorflow as tf
import scipy.io
import tensorly as ts
import os
import matplotlib.pyplot as plt
os.chdir("G:/jupyter/Guangzhou-data-set")
tensor = scipy.io.loadmat('tensor.mat')
tensor = tensor['tensor']
tensor = np.array(tensor)
print(tensor.shape)
tensor = tensor.reshape(214,-1)
print(tensor.shape)

Using numpy backend.


(214, 61, 144)
(214, 8784)


### 3.2 Model implementation

In [ ]:
class GRU_D():
    def __init__(self,network_size, input_len, hidden_size,output_len):
        
        self.input_holder = tf.placeholder(tf.float32,[None,input_len*network_size])
        self.mask_holder = tf.placeholder(tf.float32,[None,input_len*network_size])
        self.base_observation_holder = tf.placeholder(tf.float32,[None,input_len*network_size]) # last observation in literature modified
        self.decay_affector = tf.placeholder(tf.float32,[None,input_len*network_size])
        self.emprical_holder = tf.placeholder(tf.float32,[None,input_len*network_size]) # modified
        self.output_holder = tf.placeholder(tf.float32,[None,output_len*network_size])
        self.input_len = input_len
        self.output_len = output_len
                
        self.input_dimensions = network_size
        self.hidden_size = hidden_size
        self.time_steps = time_steps
        
        h_size = 32
        
        # impute input
        mask = tf.eye(self.input_len*self.network_size)
        ## diagonal to make variable indepent from each other
        w_d1 = tf.multiply(mask,weight_variable([self.input_len*self.network_size,self.input_len*self.network_size]))
        b_d1 = bias_variable([self.input_len*self.network_size])
        self.input_decay = tf.exp(-tf.maximize(tf.matmul(self.decay_affector,wd1)+bd1,0))
        
        self.input_holder = tf.multiply(self.mask_holder,self.input_holder)
        + tf.multiply(1-self.mask_holder,(tf.multiply(self.input_decay,self.base_observation_holder) + tf.multiply(1-self.input_decay, self.emprical_holder)))
        
        # impute feature
        self.w_d2 = weight_variable([ self.network_size,self.hidden_size])
        self.b_d2 = bias_variable([ self.hidden_size])
      
        # Weights for input vectors of shape (input_dimensions, hidden_size)
        self.Wr = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.input_dimensions, self.hidden_size), mean=0, stddev=0.01), name='Wr')
        self.Wz = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.input_dimensions, self.hidden_size), mean=0, stddev=0.01), name='Wz')
        self.Wh = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.input_dimensions, self.hidden_size), mean=0, stddev=0.01), name='Wh')
        
        # Weights for hidden vectors of shape (hidden_size, hidden_size)
        self.Ur = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size, self.hidden_size), mean=0, stddev=0.01), name='Ur')
        self.Uz = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size, self.hidden_size), mean=0, stddev=0.01), name='Uz')
        self.Uh = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size, self.hidden_size), mean=0, stddev=0.01), name='Uh')
        
        # Biases for hidden vectors of shape (hidden_size,)
        self.br = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='br')
        self.bz = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='bz')
        self.bh = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='bh')
        
        # Define the input layer placeholder
        self.input_holder = tf.placeholder(dtype=dtype, shape=(None, time_steps, input_dimensions), name='input')
        
        self.input_holder = tf.reshape(tf.input_holder,(-1,time_steps,input_dimensions))
        self.decay_affector1 = tf.reshape(tf.decay_affector,(-1,time_steps,input_dimensions))
        
        # Put the time-dimension upfront for the scan operator
        self.x_t = tf.transpose(self.input_holder, [1, 0, 2], name='x_t')
        self.decay_affector11 = tf.transpose(self.decay_affector1, [1, 0, 2], name='d_t')
        
        # A little hack (to obtain the same shape as the input matrix) to define the initial hidden state h_0
        self.h_0 = tf.matmul(self.x_t[0, :, :], tf.zeros(dtype=dtype, shape=(input_dimensions, hidden_size)), name='h_0')
        
        # Perform the scan operator
        self.h_t_transposed = tf.scan(self.forward_pass, (self.x_t, self.decay_affector11), initializer=self.h_0, name='h_t_transposed')
        
        # Transpose the result back
        self.h_t = tf.transpose(self.h_t_transposed, [1, 0, 2], name='h_t')
        
    def train(self,):
        w_out = weight_variable([self.hidden_size*1,self.output_len*self.input_dimensions])
        b_out = bias_variable([self.output_len*self.input_dimensions])

        self.pred = tf.nn.elu(tf,matmul(self.h_t[:,-1,:],w_out)+b_out)
        
        loss = tf.reduce_mean(tf.squared_difference(pred,self.output_holder))
        
        self.trainOp = tf.train.AdadeltaOptimizer(learning_rate=0.002).minimize(loss)
    
    def forward_pass(self, h_tm1, c_t):
        """Perform a forward pass.
        
        Arguments
        ---------
        h_tm1: np.matrix
            The hidden state at the previous timestep (h_{t-1}).
        x_t: np.matrix
            The input vector.
        """
        (x_t, d_t) = c_t
        # Definitions of update gate
        z_t = tf.sigmoid(tf.matmul(x_t, self.Wz) + tf.matmul(h_tm1, self.Uz) + self.bz)
        # Definition of reset gate
        r_t = tf.sigmoid(tf.matmul(x_t, self.Wr) + tf.matmul(h_tm1, self.Ur) + self.br)
        
        # Definition of candidate actvation
        h_proposal = tf.tanh(tf.matmul(x_t, self.Wh) + tf.matmul(tf.multiply(r_t, h_tm1), self.Uh) + self.bh)
        
        # feature decay
        self.feature_decay = tf.exp(-tf.maximize(tf.matmul(d_t,self.wd2)+self.bd2,0))
        
        h_proposal_ = tf.multiply(self.feature_decay, h_proposal)
        
        # Compute the actual next hidden state
        h_t = tf.multiply(1 - z_t, h_tm1) + tf.multiply(z_t, h_proposal_)
        
        return h_t  
        

### 3.3 Test

In [ ]:
def sample(data,input_len=6,output_len=1, mr=0.2):
    input_set=[]
    mask_set=[]
    output_set=[]
    emprical_set=[] # mean set
    base_set = []
    decay_affector_set = []
    
    empriacal = np.mean(data,axis=1)
    empriacal_full =np.array([empriacal,]*input_len)
    
    for i in range(data.shape[0]):
        origin_input = data[:,i:i+input_len]
        condition = np.random.binomial(1, 1-missing_rate, np.shape(origin_input)[0]*np.shape(origin_input)[1])
        condition = np.reshape(condition,(np.shape(origin_input)[0],np.shape(origin_input)[1]))
        mask = np.where(condition<1,0.,1.)
        origin_output = data[:,i+input_len:i+input_len+output_len]
        
        
        
        sparse_input = np.transpose(np.multiply(origin_input, mask)).reshape(-1,)
        mask = np.transpose(mask).reshape(1,-1)
        
        origin_input_T = np.transpose(origin_input).reshape(-1,)
        t = origin_input_T.shape[1]-1
        
        while t>=0:
            base = []
            decay_affector = []
            for r in range(data.shape[0]):
                tt = t-r
                k = 0
                while sparse_input[tt]<=0 and tt>=0:
                    tt-=data.shape[0]
                    k+=1
                if origin_input_T[tt]<=0:
                    base.append(-1)
                else:
                    base.append(origin_input_T[tt])
                decay_affector.append(k)
                
            t-=data.shape[0]
            
        base = base[::-1]
        decay_affector = decay_affector[::-1]
        
        input_set.append(sparse_input)
        mask_set.append(mask)
        output_set.append(np.transpose(origin_output))
        emprical_set.append(empriacal_full.reshape(-1,))
        base_set.append(base)
        decay_affector_set.append(decay_affector)
        
    return input_set, output_set, mask_set, emprical_set, base_set, decay_affector_set

def generate_batch(batch_size=32, epoch=0,input_set, output_set, mask_set, emprical_set, base_set, decay_affector_set):
    input_batch = []
    output_batch = []
    mask_batch = []
    emprical_batch = []
    base_batch = []
    decay_affector_batch=[]
    
    for i in range(batch_size):
        t = epoch
        input_batch.append(input_set[t])
        output_batch.append(output_set[t])
        mask_batch.append(mask_set[t])
        emprical_batch.append(emprical_set[t])
        base_batch.append(base_set[t])
        decay_affector_batch.append(decay_affector_set[t])
        
        epoch+=1
    return input_batch, output_batch, mask_batch, emprical_batch, base_batch, decay_affector_batch

In [ ]:
tf.reset_default_graph()
sess = tf.Session()
model = GRU_D()
model.train()
sess.run(tf.global_variables_initializer())

max_episode = 10
batch_size = 32

for i in range(max_episode):
    for j in range():
        _ = sess.run()

In [53]:
def f(x, ys):
  (y1, y2) = ys
  return x + y1 * y2

a = tf.constant([1, 2, 3, 4, 5,6,7,8,9,10,11,12])
a1 = tf.reshape(a,(-1,6))
a2 = tf.reshape(a1,(-1,3,2))
a3 = tf.transpose(a2, [1, 0, 2], name='x_t')
with tf.Session() as sess:
      print(sess.run(a1))
      print(sess.run(a3[0,:,:]))

[[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]]
[[1 2]
 [7 8]]


In [42]:
tensor[:,:2]
# np.transpose(tensor[:,:2]).reshape(-1,)

array([[40.893 , 41.938 ],
       [50.319 , 51.13  ],
       [53.88  , 53.607 ],
       [37.305 , 37.501 ],
       [38.388 , 38.383 ],
       [49.13  , 47.303 ],
       [35.466 , 36.961 ],
       [57.445 , 59.567 ],
       [39.697 , 41.162 ],
       [48.646 , 48.309 ],
       [50.983 , 50.916 ],
       [42.27  , 43.874 ],
       [45.276 , 46.113 ],
       [48.706 , 49.31  ],
       [10.621 , 20.377 ],
       [38.202 , 41.944 ],
       [50.492 , 52.671 ],
       [35.824 , 35.758 ],
       [49.225 , 49.352 ],
       [49.294 , 49.224 ],
       [39.855 , 37.938 ],
       [44.002 , 42.115 ],
       [38.834 , 42.076 ],
       [41.276 , 39.868 ],
       [37.452 , 36.12  ],
       [46.714 , 47.224 ],
       [42.157 , 42.03  ],
       [45.234 , 45.608 ],
       [45.205 , 41.881 ],
       [37.552 , 39.472 ],
       [40.196 , 37.175 ],
       [42.26  , 45.563 ],
       [38.455 , 38.246 ],
       [37.342 , 39.057 ],
       [42.787 , 41.327 ],
       [45.891 , 45.874 ],
       [54.963 , 56.746 ],
 

In [32]:
a=np.mean(tensor[:,:2],axis=0)
b=np.array([a,]*3)

In [33]:
b

array([[43.80407009, 44.14160047],
       [43.80407009, 44.14160047],
       [43.80407009, 44.14160047]])

In [35]:
b.reshape(-1,)

array([43.80407009, 44.14160047, 43.80407009, 44.14160047, 43.80407009,
       44.14160047])